# EDA

Используем датасет с https://www.kaggle.com/mlg-ulb/creditcardfraud#creditcard.csv
Файл creditcard.csv поместим в рабочую директорию данного notebook.

Признак Time можно рассмотреть, как уникальный идентификатор. Анализ временных рядов мы еще не проходили. Эта колонка, наверное, полезна для определения аномалий (может быть, мошеннические операции предваряют какие-то подготовительные особые транзанкции и отслеживать корреляции каких-то транзанкций во времени было бы полезно), но что с ней делать пока непонятно. Поэтому не будем использовать данную переменную, чтобы не вносила побочных эффектов в поведение алгоритмов.

Кроме факторов PCA в dataset есть колонка Amount — объем транзанкции.

Данные размеченные: колонка Class содержит метку. Поэтому мы можем использовать алгоритмы с обучением с учителем, а не только алгоритмы распознавания образов без учителя. Также, колонка Class пригодится для оценки результатов любых типов алгоритмов.

Изучаем описательные статистики и диаграммы:
- описательные статистики: creditcard_Descriptives.html (*HTML файл был получен с помощью приложения: Jamovi*);
- гистограммы: creditcard_Histograms.pdf;
- ящики с усами: creditcard_BoxPlots.pdf.

(*PDF файлы был получены с помощью скриптов R + пакет tidyverse, проект RStudio со скриптами находится в подкаталоге: DiagramsInR.*)

## Выводы по описательным статистикам и диаграммам

Признак Time не будем использовать, чтобы модели не переобучились по ней как по уникальному идентификатору.

Пропущенных значений в колонках нет.

Процент аномалий, подсчитанный по переменной Class: 492 / (492 + 284315) = 0.17 %. Аномалии это редкое явление в данном dataset.

По гистограммам видно, что распределения достаточно симметричные, близки к нормальному (более резкие, чем нормальное).

По гистограмма и по ящикам с усами видно, что есть много выбросов — значительная часть данных находится в выбросах.

Колонки имеют сильно разный диапазон значений, поэтому нужно нормировать данные. В данных много выбросов, поэтому применим более устойчивый к выбросам метод нормализации: RobustScaler.


## Feature engineering

1. Удаляем колонку Time.
2. Остальные колонки, кроме Class, нормируем с помощью: RobustScaler.

In [6]:
import pandas as pd
from sklearn.preprocessing import RobustScaler

df = pd.read_csv(r'.\creditcard.csv')
df = df.drop(columns='Time', axis=1)

columns_for_scaling = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14",
                       "V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28",
                       "Amount"]
scaler = RobustScaler()
df[columns_for_scaling] = scaler.fit_transform(df[columns_for_scaling])

print('Ok')

Ok


Теперь разобьем набор данных на тренировочную и тестовую выборку со стратификацией по переменной-метке (Class).

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop('Class', axis=1),
    df['Class'],
    test_size=0.2,
    stratify=df['Class'],
    random_state=1)

# Save training set
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv(r'.\creditcard_train.csv', index=False)

# Save testing set
test_data = pd.concat([X_test, y_test], axis=1)
test_data.to_csv(r'.\creditcard_test.csv', index=False)

print('Ok')

Ok


- Подготовленный для тренировки моделей dataset находится в файле creditcard_train.csv
- Тестовый dataset находится в файле creditcard_test.csv

Можем переходить к моделированию.